In [22]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pytesseract
import math
from itertools import combinations
import gensim
import gensim.downloader as api

In [47]:
from gensim.models.word2vec import Word2Vec
corpus = api.load('text8')
model = Word2Vec(corpus)

In [18]:
%%time
model_glove = api.load('glove-wiki-gigaword-50')

CPU times: user 20.8 s, sys: 299 ms, total: 21.1 s
Wall time: 21.5 s


In [ ]:
neutral, red, blue, neutral, blue

In [ ]:
red, red, blue, red, red

In [ ]:
blue, blue, neutral, neutral, neutral

In [ ]:
blue, neutral, assassin, red, blue

In [ ]:
red, blue, neutral, red, red

In [90]:
%%time
codenames('words/clean_words1.jpeg', 2)

List red, blue, neutral or assassin for row 1


 neutral, red, blue, neutral, blue


List red, blue, neutral or assassin for row 2


 red, red, blue, red, red


List red, blue, neutral or assassin for row 3


...

 red, blue, neutral, red, red


What colour is your team?


 blue


The word combination is:
['string', 'staff']
The clues are:
['axiom', 'multiplication', 'hierarchy']

The word combination is:
['string', 'princess']
The clues are:
['cardinal', 'cofinality', 'masculine']

The word combination is:
['string', 'crash']
The clues are:
['valence', 'torsion', 'ax']

The word combination is:
['string', 'orange']
The clues are:
['valence', 'tonal', 'fricatives']

The word combination is:
['string', 'crane']
The clues are:
['tonal', 'diatonic', 'clarinet']

The word combination is:
['string', 'grass']
The clues are:
['tonal', 'diatonic', 'rhythmic']

The word combination is:
['string', 'nut']
The clues are:
['tonal', 'plosives', 'valence']

The word combination is:
['staff', 'princess']
The clues are:
['electors', 'magistrates', 'mazarin']

The word combination is:
['staff', 'crash']
The clues are:
['personnel', 'ss', 'squad']

The word combination is:
['staff', 'orange']
The clues are:
['secretaries', 'grandees', 'magistrates']

The word combination is:
['sta

/Users/aydinyanna/.pyenv/versions/3.6.9/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `most_similar_cosmul` (Method will be removed in 4.0.0, use self.wv.most_similar_cosmul() instead).


In [8]:
def word_recognition(words_path):
    non_proc = cv2.imread(words_path)
    non_proc[non_proc > 10] = 255 # Turn anything other than black to white to get rid of noise
    words = pytesseract.image_to_string(non_proc)
    proc = words.replace('\n\n', ',').replace('\n',',').lower().split(',')[:25]
    
    return np.asarray(proc)

In [60]:
def grid_processing():
    
    # What does the grid look like
    print('List red, blue, neutral or assassin for row 1')
    grid1 = input()
    
    print('List red, blue, neutral or assassin for row 2')
    grid2 = input()
    
    print('List red, blue, neutral or assassin for row 3')
    grid3 = input()
    
    print('List red, blue, neutral or assassin for row 4')
    grid4 = input()
    
    print('List red, blue, neutral or assassin for row 5')
    grid5 = input()
    
    print('What colour is your team?')
    team = input()
    
    grid_list = grid1 + ',' + grid2 + ',' + grid3 + ',' + grid4 + ',' + grid5
    total_grid = grid_list.replace(' ', '').split(',')
    num_grid = [1 if colour == team else -1 if colour == 'assassin' else 0 for colour in total_grid]
    
    return np.asarray(num_grid)

In [84]:
def clue_processing(words, grid, num_words):
    
    board_words = []
    clues = []
    
    pos_words = words[grid == 1]
    neg_words = words[grid == -1]
    
    count_permutations = math.factorial(len(pos_words))/math.factorial(num_words) * math.factorial(len(pos_words) - num_words)
    combs = combinations(list(pos_words), num_words)
    arr_words = []
    for i in list(combs):
        arr_words.append(list(i))
    
    # Test this
    for i in arr_words:
        options = model.most_similar_cosmul(positive=i, negative=list(neg_words), topn=3)
        opt_words = [word for word, num in options if num > 0.8]
        if len(opt_words) > 0:
            board_words.append(i)
            clues.append(opt_words)
        else:
            continue
    
    return zip(board_words, clues)

In [62]:
def codenames(board_path, num_words):
    words = word_recognition(board_path)
    grid = grid_processing()
    
    solutions = clue_processing(words, grid, num_words)
    for combination, clue in solutions:
        print('The word combination is:')
        print(combination)
        print('The clues are:')
        print(clue)
        print()

In [52]:
model.most_similar('nail')

/Users/aydinyanna/.pyenv/versions/3.6.9/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('nose', 0.7740428447723389),
 ('claw', 0.7648398876190186),
 ('rope', 0.7589991092681885),
 ('jacket', 0.758576512336731),
 ('bow', 0.7527759075164795),
 ('boots', 0.7525355815887451),
 ('broom', 0.7451478242874146),
 ('rider', 0.7404029369354248),
 ('stick', 0.7364804744720459),
 ('wrist', 0.7336670160293579)]